In [1]:
import re
import regex
import pandas as pd
import numpy as np
import emoji
import plotly.express as px 
from collections import Counter
import nltk 
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline

In [2]:
def startsWithDateAndTime(s):
    # regex pattern for date.(Works only for android. IOS Whatsapp export format is different. Will update the code soon
    #pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9][0-9]), ([0-9]+):([0-9][0-9]) (AM|PM) -'
    pattern = '^\d{1,2}/\d{1,2}/\d{1,4}, \d{1,2}:\d{1,2}\S [AaPp][Mm] -'
    
    #02/01/2021, 1:57 pm 
    
    result = re.match(pattern, s)
    #print(result)
    if result:
        return True
    return False
  
# Finds username of any given format.
def FindAuthor(s):

    patterns = [
        #'([\w]+):',                        # First Name
        #' [a-zA-Z]:',
        '[\s]?[a-zA-Z]+:',
        '[\s]?[a-zA-Z]+ [a-zA-Z]+:',
        '[\s]?[a-zA-Z]+ [a-zA-Z]+ [a-zA-Z]+:',
     #   '[a-zA-Z]+ [a-zA-Z]+:',
        ' \+([0-9]+( [0-9]+)+):',
        '\+([0-9]+( [0-9]+)+):',
        '(([+-]?(?=\.\d|\d)(?:\d+)?(?:\.?\d*))(?:[eE]([+-]?\d+))?( ([+-]?(?=\.\d|\d)(?:\d+)?(?:\.?\d*))(?:[eE]([+-]?\d+))?)+):',
        '[a-zA-Z]+[\u263a-\U0001f999]:',
        ' [a-zA-Z]+[\u263a-\U0001f999]:'
        
    #     '([\w]+[\s]+[\w]+):',              # First Name + Last Name
    #     '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # First Name + Middle Name + Last Name
    #     '([+]\d{2} \d{5} \d{5}):',         # Mobile Number (India)
    #     '([+]\d{2} \d{3} \d{3} \d{4}):',   # Mobile Number (US)
    #     '([\w]+)[\u263a-\U0001f999]+:',    # Name and Emoji              
    ]
     
    pattern = '^' + '|'.join(patterns)
    #print(pattern)
    result = re.match(pattern, s)
    #print(result)

    if result:
        return True
    return False
  
def getDataPoint(line): 
    #print(line)  
    splitLine = line.split('-') 
    #print(splitLine)
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    if FindAuthor(message): 
        splitMessage = message.split(': ') 
        author = splitMessage[0] 
        message = ' '.join(splitMessage[1:])
        #print(message)
    else:
        author = None
    return date, time, author, message



In [3]:
parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
# Upload your file here
conversationPath = './data/WhatsAppChatwithEACEA2001Batch.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    msg = fp.readline() # Skipping first line of the file because contains information related to something about end-to-end encryption
    messageBuffer = [] 
    #print(msg)
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        if not line:
            print("in break")
            break
        line = line.strip() 
        if startsWithDateAndTime(line): 
            #print(line)
            if len(messageBuffer) > 0: 
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
                #print(parsedData)
            messageBuffer.clear() 
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message)
            #print(messageBuffer)
        else:
            messageBuffer.append(line)

in break


In [ ]:
   
df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
df["Date"] = pd.to_datetime(df["Date"])

In [5]:
df.head()

,Date,Time,Author,Message
0,2015-06-23,8:15 pm,None,"+91 98817 24864 created group ""EA CEA 2001 Ba..."
1,2021-02-01,1:57 pm,None,You joined using this group's invite link
2,2021-02-01,1:58 pm,Anisha ranjith,welcome Sijesh
3,2021-02-01,2:01 pm,Anisha ranjith,Ok thanks everyone for joining the zoom call
4,2021-02-01,2:01 pm,Anisha ranjith,It was a wonderful one as always


In [6]:
df['Author'].value_counts()[:15]

 Athira CEA          2091
 Anisha ranjith      1447
 Devi CEA            1326
 Umesh CEA           1173
 Femina               804
 Deepu CEA            684
 GT Australia         494
 Rahul Engg           368
 +91 80890 89031      368
 Ratheesh Kamu        333
 +91 98473 11000      321
 SijeshP              228
 Mangal               215
 +971 50 920 3301     202
 +91 94462 26739      159
Name: Author, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11100 entries, 0 to 11099
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     11100 non-null  datetime64[ns]
 1   Time     11100 non-null  object        
 2   Author   11037 non-null  object        
 3   Message  11100 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 347.0+ KB


In [17]:
df.dropna(inplace = True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11037 entries, 2 to 11099
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      11037 non-null  datetime64[ns]
 1   Time      11037 non-null  object        
 2   Author    11037 non-null  object        
 3   Message   11037 non-null  object        
 4   emoji     11037 non-null  object        
 5   urlcount  11037 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 603.6+ KB


In [10]:
import emoji
import regex

def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)
    
    return emoji_list


total_messages = df.shape[0]
media_messages = df[df['Message'] == '<Media omitted>'].shape[0]
df["emoji"] = df["Message"].apply(split_count)
emojis = sum(df['emoji'].str.len())
URLPATTERN = r'(https?://\S+)'
df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
links = np.sum(df.urlcount)

In [21]:
print("==== Group Stats ==========")
print("Total Messages from Jan-2021  = " ,total_messages )
print("Media msgs                    = ", media_messages)
print("Emojis                        = ", emojis )
print("Links                         = ", links)

==== Group Stats ==========
Total Messages from Jan-2021  =  11100
Media msgs                    =  2107
Emojis                        =  10727
Links                         =  301


In [23]:
media_messages_df = df[df['Message'] == '<Media omitted>']
messages_df = df.drop(media_messages_df.index)

In [24]:
text = " ".join(review for review in messages_df.Message)
print ("There are {} words in all the messages.".format(len(text)))

There are 621434 words in all the messages.


In [ ]:
stopwords = set(STOPWORDS)
stopwords.update(["ra", "ga", "na", "ani", "em", "ki", "ah","ha","la","eh","ne","le"])
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)
# Display the generated image:
# the matplotlib way:

plt.figure( figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()